In [1]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from docx2txt import docx2txt  # optional, but you used Docx2txtLoader previously
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_core.documents import Document
import uuid
import json

In [3]:
# API_KEY 불러와 환경변수로 저장
load_dotenv()  # 현재 경로의 .env 로드
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')
if not os.environ['OPENAI_API_KEY']:
    raise ValueError('OPENAI_API_KEY not found in environment. set it in .env or env vars')

In [4]:
# OpenAI client (직접 chat.completions 호출용)
llm = OpenAI()

## 1. 문서 불러오기 & 텍스트 분할

In [5]:
# 문서 로드
docx_path = "../data/langchainThon/cv/경력기술서_천승우.docx"
full_text = docx2txt.process(docx_path)  # 전체 텍스트 하나로 읽힘

In [6]:
# 텍스트 나누기
text_splitter = RecursiveCharacterTextSplitter(
    separators=['\n\n', '\n'],
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    is_separator_regex=False
)
chunks = text_splitter.split_text(full_text)

In [7]:
# 문서 리스트에 metadata 추가
documents = []
for i, c in enumerate(chunks):
    metadata = {
        "source": "경력기술서_천승우.docx",
        "chunk_id": str(i),
        # 필요하면 더 많은 메타 추가 (예: section headings, page num)
    }
    documents.append(Document(page_content=c, metadata=metadata))

In [18]:
documents

[Document(metadata={'source': '경력기술서_천승우.docx', 'chunk_id': '0'}, page_content='경력기술서\n\n[개인 정보]\n\n이메일: fourleaves8@gmail.com\n\n연락처: 010-4788-7980\n\n\n\n[전문 분야]\n\n• 데이터 사이언스       \t\t • AI 엔지니어링\t\n\n• 데이터 분석           \t\t • 프롬프트 엔지니어링\n\n• 생물분자 및 화학공학 연구\n\n\n\n[요약]\n\n저는 생명·의학 데이터를 처리하고 분석 플랫폼을 구축한 경험 및 복잡한 데이터를 효율적으로 분석해 의사결정을 지원하는 역량을 갖추고 있습니다. Python 기반의 데이터 전처리, 알고리즘 고도화, 분류 및 예측 모델 개발을 통해 항암제 신규 적응증 탐색 플랫폼을 성공적으로 구현한 바 있습니다. 또한 데이터 수집·정규화·시각화 전 과정을 주도하며, 스타트업의 코스닥 상장 및 실제 제약사 프로젝트에 기여한 실무 경험을 쌓았습니다. \n\n\n\n[보유 역량 및 기술]\n\n프로그래밍 언어\n\nPython, R, SQL, Java\n\n도구 및 프레임워크'),
 Document(metadata={'source': '경력기술서_천승우.docx', 'chunk_id': '1'}, page_content='[보유 역량 및 기술]\n\n프로그래밍 언어\n\nPython, R, SQL, Java\n\n도구 및 프레임워크\n\npandas, NumPy, Matplotlib, seaborn, Plotly, scikit-learn, PyTorch, TensorFlow, RAG, LangChain, MySQL, Oracle\n\n운영체제\n\nLinux(Ubuntu), Windows, macOS\n\nPalantir Foundry\n\nPipeline Builder, Contour, Ontology(Action), Workshop, AIP, Dataset\n\n기타\n\nVim / Vi, Gi

## 2. 임베딩 생성 및 벡터 DB 생성 (Chroma)

In [21]:
# OpenAIEmbeddings
embeddings_model = OpenAIEmbeddings(model='text-embedding-3-small')

# Chroma 벡터스토어 생성 (persist_directory 지정하면 재사용 가능)
persist_dir = './chroma_resume_db'
if not os.path.exists(persist_dir):
    os.makedirs(persist_dir, exist_ok=True)

# Chroma.from_documents 을 쓰면 내부에서 임베딩 생성 후 저장
vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings_model,
    persist_directory=persist_dir,
    collection_name='resume_seungwoo'
)

## 3. Retriever 구성 (검색 옵션: simple similarity or MMR)

In [26]:
# as_retriever로 검색 방식 바꿀 수 있음
retriever = vectorstore.as_retriever(
    search_type='mmr',  # mmr로 다양성 있는 결과 (다른 옵션: "similarity")
    search_kwargs={'k': 4, 'fetch_k': 10, 'lambda_mult': 0.2}
)

In [29]:
retriever.invoke('데이터 분석')

[Document(metadata={'chunk_id': '3', 'source': '경력기술서_천승우.docx'}, page_content='수집한 샘플 전사체 데이터 정규화 및 정량적 유전자 발현량으로 변환하여 분석 가능한 데이터셋을 구축함\n\n플랫폼 대표 성공 사례\n\n중국 ㅇㅇㅇ사 개발 신규 항암제의 주 적응증 예측 성공 및 잠재 적응증 후보 추가 제안\n\n국내 유명 제약사와 협업 – 후보물질 개발 단계 참여, 개발 물질에 대한 적응증 후보 도출, 최종 약물 타깃으로 선정되어 개발 지속\n\n상장 기술평가 항목 3가지 플랫폼 중 하나로서 회사 상장에 기여\n\nPython · R · pandas · NumPy · scikit-learn · Bioinformatics tools (survival, survminer, lifelines, GSEA, etc.) · Visualization packages (Matplotlib, seaborn, Plotly, etc.)\n\n\n\n<프로젝트2 : 항암제 플랫폼 고도화 프로젝트>\n\n1) 담당 업무 및 역할 : 암종 별, 유전자 예후 예측 스코어링 알고리즘 고도화\n\n2) 프로젝트 상세 내용'),
 Document(metadata={'chunk_id': '0', 'source': '경력기술서_천승우.docx'}, page_content='경력기술서\n\n[개인 정보]\n\n이메일: fourleaves8@gmail.com\n\n연락처: 010-4788-7980\n\n\n\n[전문 분야]\n\n• 데이터 사이언스       \t\t • AI 엔지니어링\t\n\n• 데이터 분석           \t\t • 프롬프트 엔지니어링\n\n• 생물분자 및 화학공학 연구\n\n\n\n[요약]\n\n저는 생명·의학 데이터를 처리하고 분석 플랫폼을 구축한 경험 및 복잡한 데이터를 효율적으로 분석해 의사결정을 지원하는 역량을 갖추고 있습니다. Python 기반의 데이터 전처리, 알고리즘 고도화, 분류 및 예측 모델 개발을

## 4. 질의 함수: 검색 → LLM에 컨텍스트 전달 → 응답 받기

In [33]:
def build_context_from_docs(docs):
    """
    docs: list of langchain Documents (or objects with page_content and metadata)
    returns joined context string with small citations for traceability
    """
    parts = []
    for d in docs:
        mid = d.metadata if hasattr(d, "metadata") else {}
        chunk_id = mid.get("chunk_id", "unknown")
        source = mid.get("source", "")
        parts.append(f"[chunk_id: {chunk_id} | source: {source}]\n{d.page_content}")
    return "\n\n---\n\n".join(parts)

def ask_question(question, chat_history=None, max_tokens=1000, temperature=0.0):
    """
    - 검색기로 문서 추출
    - 추출 문서들을 컨텍스트로 묶어 OpenAI ChatCompletions 호출 (gpt-4o-mini)
    - chat_history: list of {"role": "...", "content": "..."} to preserve conversation (optional)
    """
    # 1) 검색
    relevant_docs = retriever.get_relevant_documents(question)  # returns Documents
    context_text = build_context_from_docs(relevant_docs)

    # 2) build messages
    messages = []
    messages.append({"role": "system", "content": SYSTEM_PROMPT})
    # include chat history to keep multi-turn context if provided
    if chat_history:
        messages.extend(chat_history)
    # supply context as part of user message (clear delimiting)
    user_msg = (
        "Use only the CONTEXT below to answer the QUESTION.\n\n"
        f"CONTEXT:\n{context_text}\n\n"
        f"QUESTION: {question}\n\n"
        "If the context doesn't contain an answer, say you don't have the info and suggest next steps.\n"
        "Please include which chunk_id(s) you used to form the answer.\n"
    )
    messages.append({"role": "user", "content": user_msg})

    # 3) call OpenAI Chat Completions (gpt-4o-mini)
    resp = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )

    # parse response
    content = resp.choices[0].message["content"]
    # attach used sources for traceability:
    used_chunk_ids = [d.metadata.get("chunk_id") for d in relevant_docs]
    return {
        "answer": content,
        "used_chunks": used_chunk_ids,
        "relevant_docs": relevant_docs,
        "raw_api_response": resp
    }

## 구현

In [1]:
import os
import streamlit as st
from langchain_openai import ChatOpenAI
from langchain.document_loaders import PyPDFLoader, Docx2txtLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_chroma import Chroma
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough
from langchain_core.output_parsers import StrOutputParser
import requests
from bs4 import BeautifulSoup

In [4]:
# 1. API KEY
from dotenv import load_dotenv
load_dotenv()
os.environ['OPENAI_API_KEY'] = os.getenv('OPENAI_API_KEY')

st.title("💬 자기소개서 작성 도우미")

# 2. 경력기술서 업로드
uploaded_file = st.file_uploader("경력기술서 업로드 (.pdf/.docx)", type=['pdf','docx'])
if uploaded_file:
    if uploaded_file.name.endswith('.pdf'):
        loader = PyPDFLoader(uploaded_file)
    else:
        loader = Docx2txtLoader(uploaded_file)
    docs = loader.load_and_split()

2025-10-24 15:42:06.693 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 15:42:06.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 15:42:06.694 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 15:42:06.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 15:42:06.695 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 15:42:06.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 15:42:06.696 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-24 15:42:06.697 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bar